In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#Considerando que la base está en un repositorio, podemos cargarla directamente
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
cv = pd.read_csv(url)  # CSV separado por comas
cv.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


# Limpieza de Datos

En esta sección vamos a revisar y a usar las entradas que nos permitan visualizar el estado de la base de datos que usaremos para esta actividad

In [3]:
print(cv.shape)
cv.head()

(289, 1147)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


In [4]:
# Revisar el tipo de dato que contiene cada columna (Número o Categorica)
cv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Columns: 1147 entries, Province/State to 3/9/23
dtypes: float64(2), int64(1143), object(2)
memory usage: 2.5+ MB


Considerando que las columnas de fechas de nuestra base de datos, se encuentra en cada columna, lo mejor es transformarla para que se vuelva en una sola y dejas las demás de manera normal, este manejo de la base de datos, se realizará con la función pivot a través de la función **melt**

In [5]:
# columnas que se conservan de la base
id_cols = ["Province/State", "Country/Region", "Lat", "Long"]

# pasar de formato wide (fechas como columnas) a long
cv = cv.melt(
    id_vars=id_cols,           
    var_name="fecha",          
    value_name="casos"     
)

cv["fecha"] = pd.to_datetime(cv["fecha"], format="%m/%d/%y")

# vista rápida
print(cv.shape)
cv.head()

(330327, 6)


,Province/State,Country/Region,Lat,Long,fecha,casos
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0
1,NaN,Albania,41.15330,20.168300,2020-01-22,0
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0


In [6]:
# Nombres de columnas
cv.columns
list(cv.columns)

['Province/State', 'Country/Region', 'Lat', 'Long', 'fecha', 'casos']

## Datos Faltantes

En esta sesión comenzamos a ver que tan completa se encuentra nuestra base de datos, y de acuerdo al resultado de información, observamos que no exiten datos nulos en ninguna de las columnas que estamos usando

In [7]:
na_count = cv.isna().sum().sort_values(ascending=False)
na_pct   = (cv.isna().mean() * 100).round(2)
resumen_na = pd.DataFrame({'NaN_count': na_count, 'NaN_%': na_pct})
print(resumen_na)

                NaN_count  NaN_%
Country/Region          0   0.00
Lat                  2286   0.69
Long                 2286   0.69
Province/State     226314  68.51
casos                   0   0.00
fecha                   0   0.00


## Datos Nulos

In [8]:
# Análisis de datos nulos
null = cv.isnull().sum()
null_percentage = (cv.isnull().sum() / len(cv)) * 100

# Crear DataFrame con resultados
missing_data = pd.DataFrame({
    'Columna': null.index,
    'Valores Faltantes': null.values,
    'Porcentaje (%)': null_percentage.values
})

# Mostrar tabla de datos faltantes
display(missing_data)

,Columna,Valores Faltantes,Porcentaje (%)
0,Province/State,226314,68.512111
1,Country/Region,0,0.000000
2,Lat,2286,0.692042
3,Long,2286,0.692042
4,fecha,0,0.000000
5,casos,0,0.000000


De acuerdo a la revisión de los datos faltantes o nulos, se observa que su mayoría se encuentran en el campo de Province/State lo cual al revisar la base de datos y su origen, se debe a que los paises no detallaban ese campo; sin embargo, considerando que para el análisis requerido en este ejercicio no se usarán las columnas donde hay datos faltantes, no será necesario retirar esta información.

In [9]:
## Filas Repetidas
print(f'Tamaño del set antes de eliminar las filas repetidas: {cv.shape}')
cv.drop_duplicates(inplace=True)
print(f'Tamaño del set después de eliminar las filas repetidas: {cv.shape}')

Tamaño del set antes de eliminar las filas repetidas: (330327, 6)
Tamaño del set después de eliminar las filas repetidas: (330327, 6)


## Análisis de Columnas

In [10]:
#Mostrar número de paises 
unique_countries = cv['Country/Region'].nunique()
print(f"Total de países/regiones únicos: {unique_countries}")

Total de países/regiones únicos: 201


In [11]:
#Mostrar número de provincias
unique_provinces = cv['Province/State'].nunique()
print(f"Total de provincias/estados únicos: {unique_provinces}")

Total de provincias/estados únicos: 91


In [12]:
#Detalle de los países con provincias
provinces_by_country = cv.groupby('Country/Region')['Province/State'].nunique().sort_values(ascending=False)
provinces_by_country = provinces_by_country[provinces_by_country > 0]
pd.set_option('display.max_rows', 100)   
display(provinces_by_country.head(10).to_frame())

,Province/State
Country/Region,
China,34
Canada,16
United Kingdom,14
France,11
Australia,8
Netherlands,4
Denmark,2
New Zealand,2


In [15]:
# Análisis de Coordenadas Geográficas

invalid_coords = cv[
    ((cv['Lat'].notna()) & ((cv['Lat'] < -90) | (cv['Lat'] > 90))) |
    ((cv['Long'].notna()) & ((cv['Long'] < -180) | (cv['Long'] > 180)))
]

if len(invalid_coords) > 0:
    print(f"**{len(invalid_coords)} registros con coordenadas fuera de rango**")
    print(invalid_coords[['Country/Region', 'Province/State', 'Lat', 'Long']].drop_duplicates().head(10))
else:
    print("Todas las coordenadas están en rangos válidos (-90/90, -180/180)")

Todas las coordenadas están en rangos válidos (-90/90, -180/180)


In [17]:
# Asegurar tipos correctos
cv['fecha'] = pd.to_datetime(cv['fecha'], errors='coerce')
cv['casos'] = pd.to_numeric(cv['casos'], errors='coerce')

# Total por país y fecha
country_daily = (cv
                 .groupby(['Country/Region', 'fecha'], as_index=False)['casos']
                 .sum())

# Última fecha disponible
last_day = country_daily['fecha'].max()

# Top 10 en el último día
top10_last = (country_daily[country_daily['fecha'] == last_day]
              .sort_values('casos', ascending=False)
              .head(10))

print("Último día:", last_day.date())
print("\nTop 10 países (acumulado al último día):")
print(top10_last[['Country/Region', 'casos']].to_string(index=False))

Último día: 2023-03-09

Top 10 países (acumulado al último día):
Country/Region     casos
            US 103802702
         India  44690738
        France  39866718
       Germany  38249060
        Brazil  37076053
         Japan  33320438
  Korea, South  30615522
         Italy  25603510
United Kingdom  24658705
        Russia  22075858


# Análisis del Ejercicio

1. ¿En cuál mes se presentó el mayor número de contagios? 
2. ¿En ese mismo mes, cuál fue el país que reportó más contagios?
3. ¿Cuál es el país con el menor número de casos reportados hasta la fecha?

## ¿En cuál mes se presentó el mayor número de contagios?

In [29]:
# Asegurar tipos
cv['fecha'] = pd.to_datetime(cv['fecha'], errors='coerce')
cv['casos'] = pd.to_numeric(cv['casos'], errors='coerce')

# 1) Agregar por país y fecha (evita doble conteo por provincias)
country_daily = (cv
                 .groupby(['Country/Region','fecha'], as_index=False)['casos']
                 .sum()
                 .sort_values(['Country/Region','fecha']))

# 2) Pasar de acumulado a casos nuevos por día
country_daily['nuevos'] = (country_daily
                           .groupby('Country/Region')['casos']
                           .diff()
                           .fillna(country_daily['casos']))
country_daily['nuevos'] = country_daily['nuevos'].clip(lower=0)  # corrige reprocesos negativos

# 3) Total global por mes y búsqueda del máximo
mensual_global = (country_daily
                  .groupby(pd.Grouper(key='fecha', freq='ME'))['nuevos']
                  .sum()
                  .rename('contagios_mes')
                  .reset_index()
                  .sort_values('contagios_mes', ascending=False))

mes_top = mensual_global.iloc[0]
print(mensual_global.head())  # vista rápida

#Para poner el mes solicitado en formato MM del AAAA
meses_es = ['enero','febrero','marzo','abril','mayo','junio',
            'julio','agosto','septiembre','octubre','noviembre','diciembre']

f = mes_top['fecha']  # timestamp del mes ganador
mes_nombre = meses_es[f.month - 1]
anio = f.year

        fecha  contagios_mes
24 2022-01-31     90511973.0
25 2022-02-28     58260095.0
26 2022-03-31     51347389.0
30 2022-07-31     29654175.0
31 2022-08-31     25725958.0


In [30]:
print(f"Mes con más contagios (global): {mes_nombre} de {anio} con {int(mes_top['contagios_mes']):,} casos")

Mes con más contagios (global): enero de 2022 con 90,511,973 casos


## ¿En ese mismo mes, cuál fue el país que reportó más contagios?

In [32]:
# Asegurar tipos
cv['fecha'] = pd.to_datetime(cv['fecha'], errors='coerce')
cv['casos'] = pd.to_numeric(cv['casos'], errors='coerce')

#Sumar por país y fecha (evita doble conteo por provincias)
country_daily = (cv
                 .groupby(['Country/Region','fecha'], as_index=False)['casos']
                 .sum()
                 .sort_values(['Country/Region','fecha']))

#Pasar de acumulado a "nuevos" diarios por país
country_daily['nuevos'] = (country_daily
                           .groupby('Country/Region')['casos']
                           .diff()
                           .fillna(country_daily['casos']))
country_daily['nuevos'] = country_daily['nuevos'].clip(lower=0)

# 3) Filtrar enero de 2022
mask_ene22 = (country_daily['fecha'] >= '2022-01-01') & (country_daily['fecha'] < '2022-02-01')
ene22 = country_daily.loc[mask_ene22]

# 4) País con más contagios en ese mes (y top 10)
top_ene22 = (ene22.groupby('Country/Region', as_index=False)['nuevos']
             .sum()
             .sort_values('nuevos', ascending=False))


print("\nTop 10 países en enero de 2022:")
print(top_ene22.head(10).to_string(index=False))


Top 10 países en enero de 2022:
Country/Region     nuevos
            US 20336435.0
        France  9188713.0
         India  6607920.0
         Italy  4857433.0
United Kingdom  4420003.0
         Spain  3666508.0
        Brazil  3171691.0
     Argentina  2724248.0
       Germany  2706351.0
     Australia  2181746.0


In [28]:
pais_top = top_ene22.iloc[0]
print(f"País con más contagios en enero de 2022: {pais_top['Country/Region']} "
      f"con {int(pais_top['nuevos']):,} casos")

País con más contagios en enero de 2022: US con 20,336,435 casos


## ¿Cuál es el país con el menor número de casos reportados hasta la fecha?

In [33]:
# Tipos correctos
cv['fecha'] = pd.to_datetime(cv['fecha'], errors='coerce')
cv['casos'] = pd.to_numeric(cv['casos'], errors='coerce')

# Sumar por país y fecha (acumulado diario por país)
country_daily = (cv
                 .groupby(['Country/Region','fecha'], as_index=False)['casos']
                 .sum())

# Última fecha disponible en toda la base
last_day = country_daily['fecha'].max()

# Totales acumulados por país al último día
totales_last = (country_daily[country_daily['fecha'] == last_day]
                .sort_values('casos', ascending=True))

# País con MENOR número de casos (y posibles empates)
min_casos = totales_last['casos'].min()
paises_min = totales_last[totales_last['casos'] == min_casos]

# Paises con menos casos de COVID
print("\nBottom 10 países al último día:")
print(totales_last.head(10)[['Country/Region','casos']].to_string(index=False))


Bottom 10 países al último día:
      Country/Region  casos
        Korea, North      1
          MS Zaandam      9
          Antarctica     11
            Holy See     29
Winter Olympics 2022    535
    Diamond Princess    712
Summer Olympics 2020    865
              Tuvalu   2805
            Kiribati   5014
               Nauru   5247


In [34]:
print("Último día:", last_day.date())
print("\nPaís(es) con menor número de casos acumulados al último día:")
print(paises_min[['Country/Region','casos']].to_string(index=False))

Último día: 2023-03-09

País(es) con menor número de casos acumulados al último día:
Country/Region  casos
  Korea, North      1
